## **1. Install and import bibraries**

In [ ]:
!pip install -qq transformers[sentencepiece]==4.35.2 datasets==2.16.1 evaluate==0.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


In [ ]:
!sudo apt-get install libomp-dev
!pip install -qq faiss-gpu

In [ ]:
import numpy as np
import collections
import torch
import faiss
import evaluate

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
from tqdm.auto import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## **2. Download dataset**

In [ ]:
DATASET_NAME = 'squad_v2'
raw_datasets = load_dataset(DATASET_NAME, split ='train+validation')
raw_datasets

## **3. Filter out non-answerable samples**

In [ ]:
# Lọc các tập dữ liệu
raw_datasets = raw_datasets.filter(
    lambda x: len(x['answers']['text']) > 0
)

## **4. Intialize pre-trained model**

In [ ]:
# Đặt tên mô hình
MODEL_NAME = "distilbert-base-uncased"

# Khởi tạo tokenizer từ mô hình đã được huấn luyện
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Khởi tạo mô hình từ mô hình đã được huấn luyện
model = AutoModel.from_pretrained(MODEL_NAME).to(device)

## **5. Create get vector embedding functions**

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
EMBEDDING_COLUMN = 'question_embedding'
embeddings_dataset = raw_datasets.map(
    lambda x: {
        EMBEDDING_COLUMN: get_embeddings(x['question']).detach().cpu().numpy()[0]
    }
)

In [ ]:
embeddings_dataset.add_faiss_index(column=EMBEDDING_COLUMN)

## **6. Search similar samples with a question**

In [ ]:
question = 'When did Beyonce start becoming popular?'

input_quest_embedding = get_embeddings([question]).cpu().detach().numpy()
input_quest_embedding.shape

In [ ]:
TOP_K = 5
scores, samples = embeddings_dataset.get_nearest_examples(
    EMBEDDING_COLUMN, input_quest_embedding, k=TOP_K
)

In [ ]:
for idx, score in enumerate(scores):
    print(f'Top {idx + 1}\tScore: {score}')
    print(f'Question: {samples["question"][idx]}')
    print(f'Context: {samples["context"][idx]}')
    print(f'Answer: {samples["answers"][idx]}')
    print()